### Load and preprocess data

In [2]:
import scipy.io
import numpy as np
import pandas as pd

mat = scipy.io.loadmat('../Neural-Graph-Machine-Experiment/BRCA1View20000.mat')
mat = {k:v for k, v in mat.items() if k[0] != '_'}
print(len(mat['targets']))
#mat = {k: pd.Series(v[0]) for k, v in mat.items() if len(v) == 0}
mat = {k: pd.Series(v.flatten()) for k, v in mat.items()}

targets = pd.DataFrame.from_dict(mat).dropna()[['id', 'targets']]

2133


In [3]:
targets

,id,targets
0,2080.0,6.0
1,236.0,4.0
2,741.0,4.0
3,380.0,5.0
4,1446.0,3.0
...,...,...
2128,1880.0,3.0
2129,858.0,5.0
2130,698.0,2.0
2131,1626.0,5.0


In [4]:
import pandas as pd
import io
import requests
url="https://metabric.s3-us-west-1.amazonaws.com/cat2vec/cat2vec_adj.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [5]:
c['id'] = c['Patient ID'].apply(lambda x: float(x.split("-")[1]))

In [7]:
data = c.merge(targets, on='id')

In [8]:
data

,Patient ID,MB-0002,MB-0005,MB-0006,MB-0008,MB-0010,MB-0014,MB-0020,MB-0022,MB-0028,...,MB-7292,MB-7293,MB-7294,MB-7295,MB-7296,MB-7297,MB-7298,MB-7299,id,targets
0,MB-0002,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,...,-0.720671,-0.716318,-0.727589,-0.689906,-0.714645,-0.685676,-0.285254,-0.367773,2.0,5.0
1,MB-0005,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,...,0.124283,0.078778,0.049550,0.081636,0.056126,0.119477,0.023581,0.294524,5.0,4.0
2,MB-0006,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,...,0.370188,0.358792,0.361032,0.400041,0.332867,0.401476,-0.217995,0.577451,6.0,2.0
3,MB-0008,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,...,0.066668,0.043555,-0.005104,0.041898,0.037544,0.076997,0.045441,0.263049,8.0,3.0
4,MB-0010,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,...,-0.122636,-0.145899,-0.112173,-0.087051,-0.170112,-0.139419,-0.726025,0.322615,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,MB-0899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,...,0.433729,0.435154,0.423822,0.484754,0.411553,0.445852,-0.226774,0.656499,899.0,4.0
490,MB-0901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,...,0.175758,0.184256,0.165979,0.167945,0.178024,0.222212,0.482043,-0.190859,901.0,1.0
491,MB-0904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,...,-0.216419,-0.236150,-0.187077,-0.185078,-0.284451,-0.221002,-0.722680,0.235831,904.0,1.0
492,MB-0906,-0.358780,0.316354,0.633718,0.272523,0.355880,0.660196,-0.188703,-0.357670,-0.469261,...,-0.439664,-0.425679,-0.408315,-0.455820,-0.368534,-0.435420,0.294628,-0.781983,906.0,3.0


# Baseline Neural Network Implementation

In [119]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from numpy import genfromtxt
import scipy
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[data.columns[1:-2]], data['targets'], test_size=0.33, random_state=42)
X_train = torch.tensor(X_train.values).float()
y_train = torch.tensor(y_train.values).float()

In [120]:
X_test = torch.tensor(X_test.values).float()
y_test = torch.tensor(y_test.values).float()

In [121]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
  def __init__(self, values, labels):
    super(MyDataset, self).__init__()
    self.values = values
    self.labels = labels

  def __len__(self):
    return len(self.values)  # number of samples in the dataset

  def __getitem__(self, index):
    return self.values[index], self.labels[index]

In [185]:
from torch.utils.data import DataLoader
dataset = MyDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size)
testset = MyDataset(X_test, y_test)
testloader = DataLoader(testset, batch_size=5, shuffle=True)

### Neural Network related parameters

In [186]:
input_size = 1917   # The image size = 1x10312 = 10312
hidden_size = 50       # The number of nodes at the hidden layer
num_classes = 6       # The number of output classes. In this case, from 1 to 39
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 5       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence
train_test_split = 0.8

In [187]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 10312 (input data) -> 50 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 50 (hidden node) -> 39 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [188]:
model = Net(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [192]:
for epoch in range(num_epochs):
#for epoch in range(0,1):
    for i, (images, labels) in enumerate(dataloader):   # Load a batch of images with its (index, data, class)
        images = Variable(images)         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        #print(images.shape)
        #print(labels)
        #print(i)
        labels = [i-1 for i in labels]
        #print(torch.tensor(labels).reshape(-1, 1).flatten())
        labels = Variable(torch.tensor(labels).reshape(-1, 1).flatten())
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = model(images.float())                             # Forward pass: compute the output class given a image
        #if epoch == num_epochs-1:
        #    print(outputs, labels.long())
        loss = criterion(outputs, labels.long())                 # Compute the loss: difference between the output class and the pre-given label
        
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes

        if (i+1) % 20 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(dataset)//batch_size, loss.item()))

Epoch [1/5], Step [20/66], Loss: 1.6189
Epoch [2/5], Step [20/66], Loss: 1.6227
Epoch [3/5], Step [20/66], Loss: 1.6009
Epoch [4/5], Step [20/66], Loss: 1.6205
Epoch [5/5], Step [20/66], Loss: 1.6015


In [193]:
correct = 0
total = 0
all_predicted = []
all_labels = []
for i, (images, labels) in enumerate(testloader):
    images = Variable(images)
    outputs = model(images.float()) 
    labels = [i-1 for i in labels]
    labels = torch.tensor(labels).reshape(-1, 1).flatten()
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    all_predicted += list(predicted)
    #all_labels += (labels)
    
    
print('Accuracy: %d %%' % (100 * correct / total))

Accuracy: 34 %


### Split data into train/test set